# IRWA Final Project 

This project aims to build a search engine implementing different indexing and ranking algorithms. This will be done using a file containing a set of tweets from the World Health Organization (@WHO).

It will be divided in four parts:

    1) Text processing
    2) Indexing and ranking
    3) Evaluation 
    4) User Interface and Web analytics


Students Group 9:
- Mireia Beltran (U161808)
- Cisco Orteu (U162354)
- Laura Casanovas (U161832)

#### Packages

We first import all the packages needed for text processing. 

In [1]:
# if you do not have 'nltk', the following command should work "python -m pip install nltk"
import nltk
nltk.download('stopwords')
from collections import defaultdict
from array import array
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import math
import numpy as np
import collections
from numpy import linalg as la
import json
import regex as re 
import string

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mire2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Load data into memory

The dataset is stored in a txt file ```dataset_tweets_WHO.txt```and it contains a set of tweets in json format. We create tweets_data by using json.loads() function, which from a JSON string it can be parsed and it returns the content of the file.

In [6]:
text = open('dataset_tweets_WHO.txt', 'r')

In [7]:
tweets_data=[]
for line in text:
    tweet=json.loads(line)
    tweets_data.append(tweet)
text.close()

#### Dataset Creation

In order to read and process each tweet, we create a dictionary in which we will have just one row, and each column will contain one tweet

In [8]:
import pandas as pd

# Build DataFrame of tweet texts and languages
df = pd.DataFrame(tweets_data)
df.to_csv('df.csv')

We now create a new variable called 'texts' which will contain in each position of the array a tweet. Below this cell we print as an example the content of the first position of the array

In [9]:
texts=[]
for i in df:
    line =  df[i].item()['full_text']
    texts.append(str(line))

In [10]:
texts[0]

"It's International Day for Disaster Risk Reduction\n\n#OpenWHO has launched a multi-tiered core curriculum to help equip you with the competencies needed to work within public health emergency response.\n\nStart learning today &amp; be #Ready4Response:\n👉 https://t.co/hBFFOF0xKL https://t.co/fgZY22RWuS"

## 1) Text Processing

We implement the function ```build_terms(text)```.

It takes as input a text and performs the following operations:

- Stem terms
- Remove stop words
- Remove punctuation 
- Remove links
- Remove emojis
- Transform all text to lowercase
- Tokenize the text to get a list of terms

(We decided not to remove hashtags since it may be interesting to treat them separately later)

In [11]:
def build_terms(text):
    """
    Preprocess the article text (title + body) removing stop words, stemming,
    transforming in lowercase and return the tokens of the text.
    
    Argument:
    text -- string (text) to be preprocessed
    
    Returns:
    text - a list of tokens corresponding to the input text after the preprocessing
    """
    # create the pattern
    stemmer = PorterStemmer()
    
    stop_words = set(stopwords.words("english"))
    remove = string.punctuation
    remove = remove.replace("#", "–")# don't remove hashtags
    remove = remove+'¿'
    pattern = r"[{}]".format(remove) # create the pattern
    text = re.sub(pattern, "", text)
    text = re.sub(r'http\S+', '', text)
    
    #compile a regular expression pattern into a regular expression object
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    #Return the string obtained by replacing the leftmost non-overlapping occurrences of pattern in string by the replacement
    text = emoji_pattern.sub(r'', text) # no emoji
    
    # Transform in lowercase
    text=  str.lower(text) 
    # Tokenize the text to get a list of terms
    text=  text.split() 
    # Eliminate the stopwords
    text=[l for l in text if l not in stop_words] 
    

    # Perform stemming 
    text=[stemmer.stem(word) for word in text]
    
    return text

In [12]:
texts_processed = []
for i in range(len(texts)):
    texts_processed.append(build_terms(texts[i]))
    

In [27]:
def create_index(texts):
    """
    Implement the inverted index
    
    Argument:
    lines -- collection of Wikipedia articles
    
    Returns:
    index - the inverted index (implemented through a Python dictionary) containing terms as keys and the corresponding
    list of documents where these keys appears in (and the positions) as values.
    """
    index = defaultdict(list)
    title_index = {}  # dictionary to map page titles to page ids
    tweet_id = 0
    for text in texts:  # Remember, lines contain all documents
        terms = build_terms(text) #page_title + page_text
        tweet_id += 1
        title_index[tweet_id]=text  
        
        ## ===============================================================        
        ## create the index for the current page and store it in current_page_index (current_page_index)
        ## current_page_index ==> { ‘term1’: [current_doc, [list of positions]], ...,‘term_n’: [current_doc, [list of positions]]}

        ## Example: if the curr_doc has id 1 and his text is 
        ##"web retrieval information retrieval":

        ## current_page_index ==> { ‘web’: [1, [0]], ‘retrieval’: [1, [1,4]], ‘information’: [1, [2]]}

        ## the term ‘web’ appears in document 1 in positions 0, 
        ## the term ‘retrieval’ appears in document 1 in positions 1 and 4
        ## ===============================================================

        current_page_index = {}

        for term in enumerate(terms): # terms contains page_title + page_text. Loop over all terms
            try:
                # if the term is already in the index for the current page (current_page_index)
                # append the position to the corresponding list
                
        ## START CODE
                current_page_index[term][1].append(tweet_id)  
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                current_page_index[term]=tweet_id #'I' indicates unsigned int (int in Python)
            
        #merge the current page index with the main index
        for term_page, posting_page in current_page_index.items():
            index[term_page].append(posting_page)
        
        ## END CODE                    
                    
    return index, title_index

In [28]:
index, title_index= create_index(texts)